Material reference: [Feed-Forward Neural Network (FFNN) — PyTorch](https://medium.com/@carlosrodrigo.coelho/feed-forward-neural-network-ffnn-pytorch-d5d9759f53d2) by Carlos Rodrigo Coelho

Putting what we have learnt from the exploration notebook together to solve digit-recognition with a feedforward neural network.

In [ ]:
import numpy as np

import torch
import torch.nn as nn
from torchvision.datasets import MNIST
import torchvision.transforms as transforms

import matplotlib.pyplot as plt

from helper import *


Load training, testing sets. Initialize dataloaders.

In [ ]:
# load training set (no need to download)
mnist_train = MNIST(root='data/', train=True, download=False, 
                    transform=transforms.ToTensor()) 
                    # needs to transform img>tensor
# load testing set
mnist_test = MNIST(root='data/', train=False, download=False,
                   transform=transforms.ToTensor()) 
# note pytorch already partitions the dataset into training and testing sets
# obviously we can re-partition if we like


Define variables

In [ ]:
n_inputs = 28**2 # treat each pixel as an input/feature
n_classes = 10 # 0-9

# configuring nn
n_hidden = 30 # number of hidden units
batch_size = 256 # number of samples in one fw/bw pass
num_epochs = 10 # number of times we go through the dataset
learning_rate = 0.0005

In [ ]:
# use pytorch DataLoaders to automatically load/feed data during train/test
train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size, 
                                          shuffle=False)

Define FFNN

In [ ]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, n_inputs, n_hidden, n_classes):
        super(NeuralNet,self).__init__()
        self.l1 = nn.Linear(n_inputs, n_hidden) # first layer: inputs to hidden
        self.relu = nn.ReLU() # activation function (ReLU)
        self.l2 = nn.Linear(n_hidden, n_classes) # hidden to output

    def forward(self,x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
    
        return out

In [ ]:
# initialize instance    
model = NeuralNet(n_inputs, n_hidden, n_classes)

# loss and optimizer
# makes sense to use cross entropy loss because we are training for discrete
# category classifications (also could write our own loss function)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# training loop
n_total_steps = len(train_loader) # number of batches in training set

# for storing lost history
loss_hist = []

for epoch in range(num_epochs):
    epoch_loss = []
    for i, (images, labels) in enumerate(train_loader):
        # reshape images to (batch_size, input_size)
        # 100,1,28,28 -> 100,784
        images = images.reshape(-1, 28**2)
        labels = labels

        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        # saving the loss at current pass to log
        epoch_loss.append(loss.detach().numpy()) # save value

        # backward pass
        optimizer.zero_grad() # set gradients to zero in every batch
        loss.backward()     # backpropagation
        optimizer.step()   # update weights

        if (i+1) % 100 == 0: # print every 100 steps
            print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')
    
    # save mean loss of the epoch
    loss_hist.append(np.mean(np.array(epoch_loss)))

print('Finished training')

In [ ]:
plt.plot(loss_hist, "o")

In [ ]:
# test the model
with torch.no_grad(): # we don't need gradients in the testing phase
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28**2)  # 100,1,28,28 -> 100,784
        labels = labels          # 100,1 -> 100
        outputs = model(images)  # 100,10

        # value, index
        _, predictions = torch.max(outputs, 1) # 1 is the dimension
        n_samples += labels.shape[0] # number of samples in the current batch
        n_correct += (predictions == labels).sum().item()  # number of correct predictions

    acc = 100.0 * n_correct / n_samples  # accuracy
    print(f'accuracy = {acc}')